In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import io
import sys
import datetime
import math

In [ ]:
out = xr.open_dataset('../tilden/output/template_output_1hr_timestep.nc')
